In [1]:
import boto3

sqs = boto3.client('sqs')

In [2]:
response = sqs.get_queue_url(QueueName='highlighter.fifo')
queue_url = response["QueueUrl"]

In [8]:
import datetime

print(type(datetime.datetime.now()))

<class 'datetime.datetime'>


In [9]:
response = sqs.send_message(
    QueueUrl=queue_url,
    MessageAttributes={
        "Timestamp": {
            "DataType": "String",
            "StringValue": str(datetime.datetime.now())
        }
    },
    MessageBody=(
        "Information about current NY Times fiction bestseller for "
        "week of 12/11/2016."
    ),
    MessageGroupId="1",
    MessageDeduplicationId="1",
)

In [10]:
print(response["MessageId"])

f05218f2-da9f-43ed-8aee-43aeac3889dd


In [33]:
response = sqs.receive_message(
    QueueUrl=queue_url,
    AttributeNames=[
        'All'
    ],
    MaxNumberOfMessages=1,
    MessageAttributeNames=[
        'All'
    ],
    VisibilityTimeout=100,
    WaitTimeSeconds=0
)

In [41]:
print(response)

{'ResponseMetadata': {'RequestId': 'fb7f413f-5d93-55fc-a5fb-1f8dc3ad200e', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'fb7f413f-5d93-55fc-a5fb-1f8dc3ad200e', 'date': 'Mon, 06 May 2024 08:59:42 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '2', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}


In [ ]:
message = response['Messages'][0]
receipt_handle = message['ReceiptHandle']

# Delete received message from queue
sqs.delete_message(
    QueueUrl=queue_url,
    ReceiptHandle=receipt_handle
)
print('Received and deleted message: %s' % message)

In [24]:
print(message['Attributes']['SentTimestamp'])

1714985117254


In [44]:
import time

# `time.time()`: Epoch time in seconds
# `SentTimestamp`: Epoch time in milliseconds
print(time.time())
print(time.time() - float(message['Attributes']['SentTimestamp']) / 1000)

1714986409.274139
1292.0202059745789


In [19]:
print(message['MessageAttributes']['Timestamp']['StringValue'])

2024-05-06 17:45:17.110932


In [14]:
print(message['Body'])

Information about current NY Times fiction bestseller for week of 12/11/2016.


In [58]:
sqs.purge_queue(QueueUrl=queue_url)

{'ResponseMetadata': {'RequestId': '9e8fc2bc-b1ae-5b4c-ac72-9c120990bc99',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '9e8fc2bc-b1ae-5b4c-ac72-9c120990bc99',
   'date': 'Fri, 03 May 2024 10:01:34 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [41]:
import base64
a = base64.b64decode(message["Body"].encode())

In [70]:
with open("vid.ts", "wb") as video_file:
    while True:
        response = sqs.receive_message(
            QueueUrl=queue_url,
            AttributeNames=[
                "SentTimestamp"
            ],
            MaxNumberOfMessages=1,
            MessageAttributeNames=[
                "All"
            ],
            VisibilityTimeout=100,
            WaitTimeSeconds=0 # Short polling
        )

        # Non-empty response
        if "Messages" in response:
            message = response["Messages"][0]
            receipt_handle = message["ReceiptHandle"]

            sent_timestamp = message['Attributes']['SentTimestamp']

            # Delete received message from queue
            sqs.delete_message(
                QueueUrl=queue_url,
                ReceiptHandle=receipt_handle
            )

            # TODO: Replace time.time() to given value from Kafka
            # trigger_time = ...
            if time.time() - float(sent_timestamp) / 1000 > 30:
                pass
            else:
                body = base64.b64decode(message["Body"].encode())
                video_file.write(body)
        else:
            # TODO: Check emptiness of SQS
            break